In [ ]:
import torch
import torchvision.models as models
from model.autoencoder import Autoencoder
from torch import nn
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import DataLoader
from tqdm import tqdm
from torchvision import transforms as T
from data_path import DATA_PATH
from dataset import ImagesDataset, ZipDataset, VideoDataset, SampleDataset,ImagesDataset_addname
import matplotlib.pyplot as plt
from sklearn.decomposition import KernelPCA
from sklearn.manifold import TSNE
from sklearn.cluster import MiniBatchKMeans
import numpy as np
import pandas as pd
# 加载预训练的 ResNet 模型
resnet = models.resnet50(pretrained=True)

In [ ]:

dataset_train_bg = ImagesDataset_addname('./evaldata/bgr', mode='RGB', transforms=T.Compose([
    T.Resize((512, 512)),
    T.ToTensor()
]))
dataloader_train_bg = DataLoader(dataset_train_bg,
                                 shuffle=False,
                                 batch_size=4,
                                 num_workers=0,
                                 pin_memory=True)



In [ ]:
def inference(dataloader, model, batch_size=64):
    latents = []
    names=[]
    for i, (x,name) in enumerate(dataloader_train_bg):
        x = torch.FloatTensor(x)
        vec, img = model(x.cuda())
        # import pdb;pdb.set_trace()
        for j in range(0,len(name)):
            names.append(name[j])
        if i == 0:
            latents = vec.view(img.size()[0], -1).cpu().detach().numpy()
        else:
            latents = np.concatenate((latents, vec.view(img.size()[0], -1).cpu().detach().numpy()), axis=0)
    print('Latents Shape:', latents.shape)
    return latents,names

In [ ]:

resnet.eval()

# 預測答案
latents,names = inference(dataset_train_bg, model=resnet)
transformer = KernelPCA(n_components=3, kernel='rbf', n_jobs=-1)
kpca = transformer.fit_transform(latents)
